In [3]:
import azure.core
print(azure.core.VERSION)

1.26.1


In [5]:
!pip install azure-ai-ml

     |████████████████████████████████| 4.0 MB 4.1 MB/s eta 0:00:01
     |████████████████████████████████| 245 kB 51.4 MB/s eta 0:00:01
     |████████████████████████████████| 130 kB 53.1 MB/s eta 0:00:01
     |████████████████████████████████| 84 kB 3.6 MB/s  eta 0:00:01
     |████████████████████████████████| 258 kB 44.0 MB/s eta 0:00:01
     |████████████████████████████████| 49 kB 3.3 MB/s  eta 0:00:01
  Created wheel for strictyaml: filename=strictyaml-1.6.2-py3-none-any.whl size=123923 sha256=16c5586f9da2b91b116468f3fb06d6fdaa71a3585938878045a442d5b179459f
  Stored in directory: /home/azureuser/.cache/pip/wheels/e9/0b/fc/5beda6bad2ff803e820e157845679794a18f83e442da1e9f4d
Successfully built strictyaml
ERROR: azure-storage-file-datalake 12.10.1 has requirement azure-storage-blob<13.0.0,>=12.15.0, but you'll have azure-storage-blob 12.13.0 which is incompatible.


In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-227796
Azure region: southcentralus
Subscription id: a0a76bad-11a1-4a2d-9887-97a29122c8ed
Resource group: aml-quickstarts-227796


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute

cluster_name = "auto-ml"

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ##################################################
from azureml.core.compute_target import ComputeTargetException

# Define the cluster name and VM size
vm_size = "Standard_D2_V2"

# Check if the cluster already exists, and create it if it doesn't
try:
    compute_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print("Using existing compute cluster.")
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size=vm_size, max_nodes=4)
    compute_cluster = ComputeTarget.create(ws, cluster_name, compute_config)

compute_cluster.wait_for_completion(show_output=True)
### YOUR CODE HERE ##########################################

Using existing compute cluster.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [9]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler  ### YOUR CODE HERE start #######################

ps = RandomParameterSampling({
    "--C": choice(0.1, 1, 10),
    "--max_iter": choice(50, 100, 150),
})

### YOUR CODE HERE end #######################

# Specify a Policy 
### YOUR CODE HERE start ##################################################

policy = BanditPolicy(evaluation_interval=1, slack_factor=0.1)

### YOUR CODE HERE end ##################################################



if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
### YOUR CODE HERE start #########################################

src = ScriptRunConfig(
    source_directory="./",
    script="train.py",
    compute_target="auto-ml",
    environment=sklearn_env
)

### YOUR CODE HERE end ########################################


# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy. 
### YOUR CODE HERE start ##########################################################

# Define the primary metric for the hyperdrive run
primary_metric = "accuracy"

# Define the HyperDriveConfig object
hyperdrive_config = HyperDriveConfig(
    run_config=src,
    hyperparameter_sampling=ps,
    primary_metric_name=primary_metric,
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    max_total_runs=10,
    max_concurrent_runs=4,
    policy=policy
)

### YOUR CODE HERE end ######################################################

In [10]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE start ########################
from azureml.widgets import RunDetails

# Submit the hyperdrive run to the experiment
hyperdrive_run = exp.submit(hyperdrive_config, show_output=True)

# Use RunDetails to display the progress of the run
RunDetails(hyperdrive_run).show()

### YOUR CODE HERE end ##########################

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [11]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE start ##############################################
from azureml.core.run import Run

# Get the best run based on the primary metric
best_run = hyperdrive_run.get_best_run_by_primary_metric()

# Save the model from the best run
model = best_run.register_model(model_name='hyperdrive_model', model_path='outputs/hyperdrive_model.joblib')

# Download the model file to your local machine
model_file = model.download(exist_ok=True)

# Print the path to the downloaded file
print(f"Model file downloaded at {model_file}")

### YOUR CODE HERE end ###

Model file downloaded at hyperdrive_model.joblib


In [12]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE start ##################################################

from azureml.core import Dataset
data_url = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
ds = Dataset.Tabular.from_delimited_files(path=data_url)

### YOUR CODE HERE end ####################################################

In [13]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)  ### <- YOUR DATA OBJECT HERE #####################

In [14]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric= "accuracy",
    training_data=ds,
    label_column_name='y',
    n_cross_validations=5
)


In [2]:
# Submit your automl run

### YOUR CODE HERE ###
from azureml.core.experiment import Experiment
experiment_name = 'automl-bankmarketing'
experiment = Experiment(ws, experiment_name)
run = experiment.submit(automl_config, show_output=True)

In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
# Monitor the run
RunDetails(run).show()

# Retrieve the best model and save it
best_run, fitted_model = run.get_output()
model_name = best_run.properties['model_name']
joblib.dump(fitted_model, './outputs/' + model_name + '.pkl')